<a href="https://colab.research.google.com/github/Anthonyvijay10/AI-Training/blob/main/Transformers/Fine_tune_Perceiver_for_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up environment

As usual, we first install HuggingFace Transformers, and Datasets.

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.3 MB/s eta 0:00:00


In [2]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Prepare data

Here we take a small portion of the IMDB dataset, a binary text classification dataset ("is a movie review positive or negative?").

In [3]:
from datasets import load_dataset

train_ds, test_ds = load_dataset("imdb", split=['train[:10]+train[-10:]', 'test[:5]+test[-5:]'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

We create id2label and label2id mappings, which are handy at inference time.

In [4]:
labels = train_ds.features['label'].names
print(labels)

['neg', 'pos']


In [5]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
print(id2label)

{0: 'neg', 1: 'pos'}


Next, we prepare the data for the model using the tokenizer.

In [6]:
from transformers import PerceiverTokenizer

tokenizer = PerceiverTokenizer.from_pretrained("deepmind/language-perceiver")

train_ds = train_ds.map(lambda examples: tokenizer(examples['text'], padding="max_length", truncation=True),
                        batched=True)
test_ds = test_ds.map(lambda examples: tokenizer(examples['text'], padding="max_length", truncation=True),
                      batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

We set the format to PyTorch tensors, and create familiar PyTorch dataloaders.

In [7]:
train_ds.set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
test_ds.set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])

In [8]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_ds, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=4)

Here we verify some things (always important to check out your data!).

In [9]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  print(k,v.shape)

label torch.Size([4])
input_ids torch.Size([4, 2048])
attention_mask torch.Size([4, 2048])


In [10]:
tokenizer.decode(batch['input_ids'][3])

"[CLS]With the mixed reviews this got I wasn't expecting too much, and was pleasantly surprised. It's a very entertaining small crime film with interesting characters, excellent portrayals, writing that's breezy without being glib, and a good pace. It looks good too, in a funky way. Apparently people either like this movie or just hate it, and I'm one who liked it.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PA

In [11]:
batch['label']

tensor([0, 0, 1, 1])

## Define model

Next, we define our model, and put it on the GPU.

In [12]:
from transformers import PerceiverForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PerceiverForSequenceClassification.from_pretrained("deepmind/language-perceiver",
                                                               num_labels=2,
                                                               id2label=id2label,
                                                               label2id=label2id)
model.to(device)

config.json:   0%|          | 0.00/911 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/804M [00:00<?, ?B/s]

Some weights of PerceiverForSequenceClassification were not initialized from the model checkpoint at deepmind/language-perceiver and are newly initialized: ['perceiver.decoder.decoder.decoding_cross_attention.attention.output.dense.bias', 'perceiver.decoder.decoder.decoding_cross_attention.attention.output.dense.weight', 'perceiver.decoder.decoder.decoding_cross_attention.attention.self.key.bias', 'perceiver.decoder.decoder.decoding_cross_attention.attention.self.key.weight', 'perceiver.decoder.decoder.decoding_cross_attention.attention.self.layernorm1.bias', 'perceiver.decoder.decoder.decoding_cross_attention.attention.self.layernorm1.weight', 'perceiver.decoder.decoder.decoding_cross_attention.attention.self.layernorm2.bias', 'perceiver.decoder.decoder.decoding_cross_attention.attention.self.layernorm2.weight', 'perceiver.decoder.decoder.decoding_cross_attention.attention.self.query.bias', 'perceiver.decoder.decoder.decoding_cross_attention.attention.self.query.weight', 'perceiver.de

PerceiverForSequenceClassification(
  (perceiver): PerceiverModel(
    (input_preprocessor): PerceiverTextPreprocessor(
      (embeddings): Embedding(262, 768)
      (position_embeddings): Embedding(2048, 768)
    )
    (embeddings): PerceiverEmbeddings()
    (encoder): PerceiverEncoder(
      (cross_attention): PerceiverLayer(
        (attention): PerceiverAttention(
          (self): PerceiverSelfAttention(
            (layernorm1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
            (layernorm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (query): Linear(in_features=1280, out_features=256, bias=True)
            (key): Linear(in_features=768, out_features=256, bias=True)
            (value): Linear(in_features=768, out_features=1280, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): PerceiverSelfOutput(
            (dense): Linear(in_features=1280, out_features=1280, bias=True)
          )
        )


## Train the model

Here we train the model using native PyTorch.

In [13]:
from transformers import AdamW
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(20):  # loop over the dataset multiple times
    print("Epoch:", epoch)
    for batch in tqdm(train_dataloader):
         # get the inputs;
         inputs = batch["input_ids"].to(device)
         attention_mask = batch["attention_mask"].to(device)
         labels = batch["label"].to(device)

         # zero the parameter gradients
         optimizer.zero_grad()

         # forward + backward + optimize
         outputs = model(inputs=inputs, attention_mask=attention_mask, labels=labels)
         loss = outputs.loss
         loss.backward()
         optimizer.step()

         # evaluate
         predictions = outputs.logits.argmax(-1).cpu().detach().numpy()
         accuracy = accuracy_score(y_true=batch["label"].numpy(), y_pred=predictions)
         print(f"Loss: {loss.item()}, Accuracy: {accuracy}")

Epoch: 0


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 0.7277336120605469, Accuracy: 0.0
Loss: 2.500199794769287, Accuracy: 0.25
Loss: 2.0635900497436523, Accuracy: 0.0
Loss: 0.686470091342926, Accuracy: 0.5
Loss: 1.6674597263336182, Accuracy: 0.0
Epoch: 1


  0%|          | 0/5 [00:00<?, ?it/s]

...

Loss: 0.14169429242610931, Accuracy: 1.0
Loss: 0.4839552342891693, Accuracy: 0.75
Loss: 0.776053786277771, Accuracy: 0.5
Loss: 0.28095486760139465, Accuracy: 1.0
Loss: 0.4102437496185303, Accuracy: 1.0
Epoch: 18


  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 0.30409592390060425, Accuracy: 0.75
Loss: 0.5624896883964539, Accuracy: 0.5
Loss: 0.5073601007461548, Accuracy: 0.75
Loss: 0.9168227910995483, Accuracy: 0.25
Loss: 0.39418065547943115, Accuracy: 0.75
Epoch: 19


  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 0.4792799651622772, Accuracy: 0.75
Loss: 0.26577720046043396, Accuracy: 1.0
Loss: 0.4208076000213623, Accuracy: 0.75
Loss: 0.7527266144752502, Accuracy: 0.5
Loss: 0.5673931837081909, Accuracy: 0.75


## Inference

In [16]:
text = "I loved this movie, it's super good."

input_ids = tokenizer(text, return_tensors="pt").input_ids

# forward pass
outputs = model(inputs=input_ids.to(device))
logits = outputs.logits
predicted_class_idx = logits.argmax(-1).item()

print("Predicted:", model.config.id2label[predicted_class_idx])

Predicted: pos
